## Część 2

In [37]:
import os
import pandas as pd
from google.cloud import bigquery
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="secret.json" # lokalizacja pobranego klucza z punktu 1.4.
client = bigquery.Client()
query = ('select * from bigquery-public-data.covid19_open_data.covid19_open_data limit 10')
query_job = client.query(query)
query_result = query_job.result()
df1 = query_result.to_dataframe()
df1

,location_key,date,place_id,wikidata_id,datacommons_id,country_code,country_name,iso_3166_1_alpha_2,iso_3166_1_alpha_3,aggregation_level,...,cumulative_vaccine_doses_administered_pfizer,new_persons_fully_vaccinated_moderna,cumulative_persons_fully_vaccinated_moderna,new_vaccine_doses_administered_moderna,cumulative_vaccine_doses_administered_moderna,new_persons_fully_vaccinated_janssen,cumulative_persons_fully_vaccinated_janssen,new_vaccine_doses_administered_janssen,cumulative_vaccine_doses_administered_janssen,location_geometry
0,AD,2022-02-16,ChIJlfCemC71pRIRkn_qeNc-yQc,Q228,country/AND,AD,Andorra,AD,AND,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(1.555278 42.558333)
1,AD,2021-12-24,ChIJlfCemC71pRIRkn_qeNc-yQc,Q228,country/AND,AD,Andorra,AD,AND,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(1.555278 42.558333)
2,AD,2021-04-04,ChIJlfCemC71pRIRkn_qeNc-yQc,Q228,country/AND,AD,Andorra,AD,AND,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(1.555278 42.558333)
3,AD,2022-07-10,ChIJlfCemC71pRIRkn_qeNc-yQc,Q228,country/AND,AD,Andorra,AD,AND,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(1.555278 42.558333)
4,AD,2021-02-17,ChIJlfCemC71pRIRkn_qeNc-yQc,Q228,country/AND,AD,Andorra,AD,AND,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(1.555278 42.558333)
5,AD,2020-04-03,ChIJlfCemC71pRIRkn_qeNc-yQc,Q228,country/AND,AD,Andorra,AD,AND,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(1.555278 42.558333)
6,AE,2020-10-03,ChIJvRKrsd9IXj4RpwoIwFYv0zM,Q878,country/ARE,AE,United Arab Emirates,AE,ARE,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(54.3 24.4)
7,AE,2020-05-25,ChIJvRKrsd9IXj4RpwoIwFYv0zM,Q878,country/ARE,AE,United Arab Emirates,AE,ARE,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(54.3 24.4)
8,AE,2021-04-13,ChIJvRKrsd9IXj4RpwoIwFYv0zM,Q878,country/ARE,AE,United Arab Emirates,AE,ARE,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(54.3 24.4)
9,AE,2021-04-08,ChIJvRKrsd9IXj4RpwoIwFYv0zM,Q878,country/ARE,AE,United Arab Emirates,AE,ARE,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(54.3 24.4)


## Część 3

3.1. Sprawdź, ile jest zapisanych wierszy z danymi. 

In [38]:
query = ("""
select count(*) as liczba_wierszy from bigquery-public-data.covid19_open_data.covid19_open_data;
""")

query_job = client.query(query)
query_result = query_job.result()
df = query_result.to_dataframe()
df

,liczba_wierszy
0,22756333


 3.2. Sprawdź, ile krajów jest uwzględnionych w danych. 



In [39]:
query = ("""
select count(*) as liczba_krajow FROM (SELECT DISTINCT country_name FROM bigquery-public-data.covid19_open_data.covid19_open_data);
""")
query_job = client.query(query)
query_result = query_job.result()
df = query_result.to_dataframe()
df

,liczba_krajow
0,246


 3.3. Sprawdź, w jaki sposób zapisywane są dzienne informacje dla krajów.



Dzienne informacje dla krajów raportowane są z podziałem na części opisywane przez aggregation_level.
Dane odnośnie osób hospitalizowanych, zaszczepionych, zmarłych i testowanych raportowane są oddzielnie dla poszczególnych przedziałów wiekowych oraz z podziałem na płeć. 
Dane zawierają również podstawowe informacje o poszczególnych krajach, takie jak populacja, powierzchnia, wilgotność czy przewidywana długość życia.

 3.4. Sprawdź, w jaki sposób zapisywane są wartości liczbowe.  



In [40]:
print(df1['population'].dtype)
print(df1['nurses_per_1000'].dtype)
print(df1['new_tested'].dtype)

Int64
float64
object


 3.5. Sprawdź, jaki przedział czasowy jest uwzględniony w danych. Dodatkowo porównaj przedziały czasowe dla przypadków nowych zachorowań, nowych śmierci oraz nowych zaszczepionych osób w danych.



In [41]:
query = ("""
select min(date) as poczatek_przedzialu, max(date) as koniec_przedzialu FROM bigquery-public-data.covid19_open_data.covid19_open_data;
""")
query_job = client.query(query)
query_result = query_job.result()
df = query_result.to_dataframe()
df
# co to znaczy nowych?

,poczatek_przedzialu,koniec_przedzialu
0,2020-01-01,2022-09-17


In [42]:
query = ("""
select min(date) as poczatek_przedzialu_zachorowan, max(date) as koniec_przedzialu_zachorowan FROM bigquery-public-data.covid19_open_data.covid19_open_data where new_confirmed is not null and new_confirmed <> 0;
""")
query_job = client.query(query)
query_result = query_job.result()
df = query_result.to_dataframe()
df

,poczatek_przedzialu_zachorowan,koniec_przedzialu_zachorowan
0,2020-01-01,2022-09-15


In [43]:
query = ("""
select min(date) as poczatek_przedzialu_smierci, max(date) as koniec_przedzialu_smierci FROM bigquery-public-data.covid19_open_data.covid19_open_data where new_deceased is not null and new_deceased <> 0;
""")
query_job = client.query(query)
query_result = query_job.result()
df = query_result.to_dataframe()
df

,poczatek_przedzialu_smierci,koniec_przedzialu_smierci
0,2020-01-02,2022-09-15


In [44]:
query = ("""
select min(date) as poczatek_przedzialu_zaszczepionych, max(date) as koniec_przedzialu_zaszczepionych FROM bigquery-public-data.covid19_open_data.covid19_open_data where new_persons_vaccinated is not null and new_persons_vaccinated <> 0;
""")
query_job = client.query(query)
query_result = query_job.result()
df = query_result.to_dataframe()
df

,poczatek_przedzialu_zaszczepionych,koniec_przedzialu_zaszczepionych
0,2020-12-07,2022-09-14


 3.6. Sprawdź więcej informacji (co najmniej 5 różnych) o danych dotyczących COVID-19. W tym celu nie wykonuj żadnych dodatkowych obliczeń. 



In [45]:
# Sprawdzenie czy wystepuja wartosci ujemne
query = ("""
select COUNT(*) FROM bigquery-public-data.covid19_open_data.covid19_open_data where new_confirmed < 0;
""")
query_job = client.query(query)
query_result = query_job.result()
df = query_result.to_dataframe()
df

,f0_
0,102199


In [46]:
#Sprawdzenie czy wystepuja duplikaty
query = ("""
select COUNT(*), location_key, date FROM bigquery-public-data.covid19_open_data.covid19_open_data group by location_key, date having count(*) > 1;
""")
query_job = client.query(query)
query_result = query_job.result()
df = query_result.to_dataframe()
df

,f0_,location_key,date


In [47]:
# Sprawdzenie czy kluczowe kolumny zawieraja nulle
query = ("""
select COUNT(*) FROM bigquery-public-data.covid19_open_data.covid19_open_data where date is null or country_name is null or location_key is null or iso_3166_1_alpha_3 is null;
""")
query_job = client.query(query)
query_result = query_job.result()
df = query_result.to_dataframe()
df

,f0_
0,991


In [48]:
# Sprawdzenie z jakich krajow posiadamy najwiecej danych
query = ("""
select country_name, count(*) as count from bigquery-public-data.covid19_open_data.covid19_open_data group by country_name order by count desc limit 10;
""")
query_job = client.query(query)
query_result = query_job.result()
df = query_result.to_dataframe()
df

,country_name,count
0,Brazil,5565456
1,United States of America,3255435
2,Mexico,2475518
3,Peru,1883891
4,Israel,1472626
5,Spain,1385418
6,Colombia,1145596
7,India,747214
8,Indonesia,544059
9,Argentina,540095


In [49]:
# sprawdzenie czy w kolumnach cumulative wystepuja wartosci mniejsze niz w kolumnach new
query = ("""
select distinct location_key, country_name from bigquery-public-data.covid19_open_data.covid19_open_data where (new_confirmed > cumulative_confirmed or new_deceased > cumulative_deceased or cast(new_tested as int) > cumulative_tested or new_persons_vaccinated > cumulative_persons_vaccinated or new_persons_fully_vaccinated > cumulative_persons_fully_vaccinated or new_vaccine_doses_administered > cumulative_vaccine_doses_administered);
""")
query_job = client.query(query)
query_result = query_job.result()
df = query_result.to_dataframe()
df

,location_key,country_name
0,PL_22,Poland
1,PL_26,Poland
2,PL_14,Poland
3,PL_16,Poland
4,PL_18,Poland
5,PL_04,Poland
6,CH_GE,Switzerland
7,US_OK,United States of America
8,PL_30,Poland
9,CL,Chile


## Część 4

4.1. Chcemy posiadać podstawowe dane o wszystkich krajach świata, które będą zrozumiałe dla człowieka oraz uniwersalne i potencjalnie przyszłościowe do dalszego przetwarzania.



In [50]:
query = ("""
SELECT #distinct?
    country_code,
    date,  
    country_name, 
    iso_3166_1_alpha_2, #imo useless
    iso_3166_1_alpha_3, 
    population, 
    population_male, 
    population_female, 
    population_rural, 
    population_urban, 
    population_density, 
    human_development_index, 
    gdp_usd, 
    gdp_per_capita_usd, 
    latitude, 
    longitude, 
    area_sq_km, 
    nurses_per_1000, #dalbym do nowego
    physicians_per_1000, #dalbym do nowego
    health_expenditure_usd, 
    out_of_pocket_health_expenditure_usd 
FROM `bigquery-public-data.covid19_open_data.covid19_open_data`
WHERE 
    country_code IS NOT NULL AND 
    date IS NOT NULL AND  
    country_name IS NOT NULL AND 
    iso_3166_1_alpha_2 IS NOT NULL AND 
    iso_3166_1_alpha_3 IS NOT NULL AND 
    population IS NOT NULL AND 
    population_male IS NOT NULL AND 
    population_female IS NOT NULL AND 
    population_rural IS NOT NULL AND 
    population_urban IS NOT NULL AND 
    population_density IS NOT NULL AND 
    human_development_index IS NOT NULL AND 
    gdp_usd IS NOT NULL AND 
    gdp_per_capita_usd IS NOT NULL AND 
    latitude IS NOT NULL AND 
    longitude IS NOT NULL AND 
    area_sq_km IS NOT NULL AND 
    nurses_per_1000 IS NOT NULL AND 
    physicians_per_1000 IS NOT NULL AND 
    health_expenditure_usd IS NOT NULL AND 
    out_of_pocket_health_expenditure_usd IS NOT NULL;
    AND aggregation_level = 0;
""")
query_job = client.query(query)
query_result = query_job.result()
df1 = query_result.to_dataframe()
df1.to_csv("countries_data.csv", index=False)
df1

 4.2. Chcemy wygenerować statystyki dotyczące zachorowań na COVID-19 na całym świecie.



In [51]:
query = ("""select distinct date, country_name, new_confirmed, cumulative_confirmed, new_tested,  cumulative_tested from bigquery-public-data.covid19_open_data.covid19_open_data WHERE
  date IS NOT NULL
  AND country_name IS NOT NULL
  AND new_confirmed IS NOT NULL
  AND cumulative_confirmed IS NOT NULL
  AND new_tested IS NOT NULL
  AND cumulative_tested IS NOT NULL""")
query_job = client.query(query)
query_result = query_job.result()
df2 = query_result.to_dataframe()
df2.to_csv("covid_cases_stats.csv", index=False)
df2

 4.3. Chcemy poznać efekty COVID-19 poprzez uwypuklenie problemu śmiertelności ludzi spowodowanej wirusem.



In [52]:
query = ("""
SELECT 
    date,  
    country_name,  
    new_deceased, 
    cumulative_deceased 
FROM `bigquery-public-data.covid19_open_data.covid19_open_data`
WHERE 
    date IS NOT NULL AND  
    country_name IS NOT NULL AND 
    new_deceased IS NOT NULL AND 
    cumulative_deceased IS NOT NULL;

""")
query_job = client.query(query)
query_result = query_job.result()
df3 = query_result.to_dataframe()
df3.to_csv("covid_mortality.csv", index=False)
df3

 4.4. Chcemy zaobserwować trendy i zależności dotyczące szczepień na COVID-19.



In [53]:
query = ("""
SELECT 
    date,  
    country_name, 
    new_persons_vaccinated, 
    cumulative_persons_vaccinated, 
    new_persons_fully_vaccinated, 
    cumulative_persons_fully_vaccinated, 
    new_vaccine_doses_administered, 
    cumulative_vaccine_doses_administered 
FROM `bigquery-public-data.covid19_open_data.covid19_open_data`
WHERE 
    date IS NOT NULL AND  
    country_name IS NOT NULL AND 
    new_persons_vaccinated IS NOT NULL AND 
    cumulative_persons_vaccinated IS NOT NULL AND 
    new_persons_fully_vaccinated IS NOT NULL AND 
    cumulative_persons_fully_vaccinated IS NOT NULL AND 
    new_vaccine_doses_administered IS NOT NULL AND 
    cumulative_vaccine_doses_administered IS NOT NULL;

""")
query_job = client.query(query)
query_result = query_job.result()
df4 = query_result.to_dataframe()
df4.to_csv("covid_vaccination.csv", index=False)
df4

 4.5. Zdefiniuj własny dodatkowy przypadek. Chcemy dowiedzieć się ile osób potrzebowało pomocy medycznej.



In [54]:
query = ("""
SELECT 
    date,  
    country_name, 
    new_hospitalized_patients,
    cumulative_hospitalized_patients,
    new_intensive_care_patients,
    cumulative_intensive_care_patients
FROM `bigquery-public-data.covid19_open_data.covid19_open_data`
WHERE 
    date IS NOT NULL AND  
    country_name IS NOT NULL AND 
    new_hospitalized_patients IS NOT NULL AND
    cumulative_hospitalized_patients IS NOT NULL AND
    new_intensive_care_patients IS NOT NULL AND
    cumulative_intensive_care_patients IS NOT NULL;
""")
query_job = client.query(query)
query_result = query_job.result()
df5 = query_result.to_dataframe()
df5.to_csv("covid_vaccination.csv", index=False)
df5

## Część 5
Połącz ze sobą wszystkie dane otrzymane w części 4. Nowy zbiór danych zapisz jako oddzielny obiekt DataFrame. Jeżeli uważasz, że należy stworzyć kilka takich obiektów, zrób to i zapisz swoje uzasadnienie. Pamiętaj o ciągłej konieczności zachowania czystości danych. Wynik tego zadania zapisz w pliku / plikach CSV.

In [56]:
merged_df = df1.merge(df2, on=["country_name", "date"], how="inner")
merged_df = merged_df.merge(df3, on=["country_name", "date"], how="inner")
merged_df = merged_df.merge(df4, on=["country_name", "date"], how="inner")
merged_df = merged_df.merge(df5, on=["country_name", "date"], how="inner")
merged_df = merged_df.dropna()
merged_df.to_csv("covid_combined_data.csv", index=False)
merged_df

MemoryError: Unable to allocate 74.5 GiB for an array with shape (10004580344,) and data type int64

## Część 6
Połącz ze sobą dane otrzymane w części 5 oraz dane znajdujące się w plikach

In [ ]:
world_countries = pd.read_csv("world_countries.csv")
gdp = pd.read_csv("GDP.csv")
merged_df2 = df1.merge(world_countries, on="iso_3166_1_alpha_3", how="inner")
# nie rozumiem troche, imo bez sensu mergowac to z tymi calymi danymi koncowymi z 5, a w GDP country name to wgl nazwa kraju lub regionu wiec troche xd